In [ ]:
%pip install transformers accelerate torch evaluate datasets peft bitsandbytes

# BLEURT score
- [Link to Repo](https://github.com/google-research/bleurt/blob/master/checkpoints.md)
- [HF Eval Page](https://huggingface.co/spaces/evaluate-metric/bleurt)

In [ ]:
# mount gdrive to save results
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/WCU_THESIS/AITA_Evaluation

/content/drive/MyDrive/WCU_THESIS/AITA_Evaluation


In [ ]:
!pip install --upgrade pip  # ensures that pip is current
!git clone https://github.com/google-research/bleurt.git
%cd /content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/bleurt
!pip install -e .

In [ ]:
# Downloads the BLEURT-base checkpoint.
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
!unzip BLEURT-20.zip

--2024-03-19 17:12:26--  https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2140294207 (2.0G) [application/octet-stream]
Saving to: ‘BLEURT-20.zip’

BLEURT-20.zip       100%[===================>]   1.99G  23.0MB/s    in 94s     

2024-03-19 17:14:00 (21.8 MB/s) - ‘BLEURT-20.zip’ saved [2140294207/2140294207]

--2024-03-19 17:14:00--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2024-03-19 17:14:00--
Total wall clock time: 1m 34s
Downloaded: 1 files, 2.0G in 1m 34s (21.8 MB/s)
Archive:  BLEURT-20.zip
   creating: BLEURT-20/
  inflating: BLEURT-20/bert_config.json  
  inflating: BLEURT-20/saved_model.pb  
   creating: BLEURT-20/variables/


In [ ]:
%cd ..

/content/drive/MyDrive/WCU_THESIS/AITA_Evaluation


In [ ]:
import json

with open('flanT5_small_evaluation/test_results.json', 'r') as file:
    test_results = json.load(file)

candidates = test_results['predictions']
references = test_results['reference_texts']

In [ ]:
# Open a file to write
with open('flanT5_small_evaluation/test_result_pairs.jsonl', 'w') as file:
    # Iterate over the zipped lists
    for ref, cand in zip(references, candidates):
        # Create a dictionary with the specified keys and values
        json_object = {'reference': ref, 'candidate': cand}
        # Convert the dictionary to a JSON string
        json_str = json.dumps(json_object)
        # Write the JSON string to the file with a newline
        file.write(json_str + '\n')

In [ ]:
import shutil
source_path = '/content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/flanT5_small_evaluation/test_result_pairs.jsonl'
destination_path = '/content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/bleurt/test_result_pairs.jsonl'
shutil.copy2(source_path, destination_path)

'/content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/bleurt/test_result_pairs.jsonl'

In [ ]:
%cd /content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/bleurt

/content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/bleurt


In [ ]:
import json

# Path to your JSONL file

# Create an empty list to store the data
test_result_pairs = []

# Open the file and read line by line
with open('test_result_pairs.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and add to the list
        test_result_pairs.append(json.loads(line))


In [ ]:
# create flan_t5_small_scores.txt
!touch flan_t5_small_scores.txt

In [ ]:
!python -m bleurt.score_files \
  -sentence_pairs_file=test_result_pairs.jsonl \
  -bleurt_checkpoint=BLEURT-20 \
  -scores_file=flan_t5_small_scores.txt

2024-03-19 19:12:52.549767: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 19:12:52.601594: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 19:12:52.601646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 19:12:52.603332: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 19:12:52.611706: I tensorflow/core/platform/cpu_feature_guar